In [2]:
#import dependecies
import pandas as pd
from pathlib import Path
import polars as pl
import re

In [3]:
df1= pd.read_csv("player_possession3.csv", encoding='unicode_escape')
df2= pd.read_csv("player_shooting3.csv", encoding='unicode_escape')
df3= pd.read_csv("player_passing3.csv", encoding='unicode_escape')
df4= pd.read_csv("player_defense3.csv", encoding='unicode_escape')
df5= pd.read_csv("FIFA_official_data3.csv", encoding='unicode_escape')

In [4]:
# Right after import, drop repeated columns from data frames to prevent duplicating upon merge.
df2 = df2.drop(columns=['age', 'team','position','minutes_90s','birth_year'])
df3 = df3.drop(columns=['age', 'team','position','minutes_90s','birth_year'])
df4 = df4.drop(columns=['age', 'team','position','minutes_90s','birth_year'])

In [5]:
df5['Value'] = df5['Value'].str[1:]
df5['Value']

0           78M
1        119.5M
2        125.5M
3           45M
4          129M
          ...  
16705       30K
16706       20K
16707       50K
16708         0
16709         0
Name: Value, Length: 16710, dtype: object

In [6]:
def conversion(value):
    if 'K' in value:
        return float(value.replace('K', '')) * 1000
    elif 'M' in value:
        return float(value.replace('M', '')) * 1000000
    else:
        return float(value)

df5['Value'] = df5['Value'].apply(conversion)

df5['Value']

0         78000000.0
1        119500000.0
2        125500000.0
3         45000000.0
4        129000000.0
            ...     
16705        30000.0
16706        20000.0
16707        50000.0
16708            0.0
16709            0.0
Name: Value, Length: 16710, dtype: float64

In [7]:
qualified_countries = ['Argentina', 'Australia', 'Belgium', 'Brazil', 'Cameroon', 'Canada', 'Costa Rica', 'Croatia', 'Denmark',
                       'Ecuador', 'England', 'France', 'Germany', 'Ghana', 'Iran', 'Japan', 'Mexico', 'Morocco', 'Netherlands', 
                       'Poland', 'Portugal', 'Qatar', 'Saudi Arabia', 'Senegal', 'Serbia', 'South Korea', 'Spain', 'Switzerland',
                      'Tunisia', 'Uruguay', 'United States', 'Wales']

df5_qualified = df5[df5['Nationality'].isin(qualified_countries)]

print(df5_qualified)

           ID              Name  Age  \
0      158023          L. Messi   34   
1      188545    R. Lewandowski   32   
2      192985      K. De Bruyne   30   
3       20801        C. Ronaldo   36   
4      190871            Neymar   29   
...       ...               ...  ...   
16702  260623      21 M. Wright   33   
16705  235352   18 T. Käßemodel   28   
16706  219735    15 T. Fletcher   19   
16707  220806      16 E. Redman   18   
16708   19334  10 I. Baraclough   38   

                                                  Photo Nationality  \
0      https://cdn.sofifa.com/players/158/023/22_60.png   Argentina   
1      https://cdn.sofifa.com/players/188/545/22_60.png      Poland   
2      https://cdn.sofifa.com/players/192/985/22_60.png     Belgium   
3      https://cdn.sofifa.com/players/020/801/22_60.png    Portugal   
4      https://cdn.sofifa.com/players/190/871/22_60.png      Brazil   
...                                                 ...         ...   
16702  https://cdn.sof

In [8]:
#Merge columns together using 'pd.merge'
merged_df = pd.merge(df1,df2,how='inner',left_on='player',right_on='player')
merged_df = pd.merge(merged_df,df3,how='inner',left_on='player',right_on='player')
merged_df = pd.merge(merged_df,df4,how='inner',left_on='player',right_on='player')

In [9]:
merged_df.head()

,player,position,team,age,birth_year,minutes_90s,touches,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,...,dribbles_vs,dribble_tackles_pct,dribbled_past,blocks,blocked_shots,blocked_passes,interceptions,tackles_interceptions,clearances,errors
0,Aaron Mooy,MF,Australia,32-094,1990,4.0,255.0,14.0,62.0,168.0,...,8.0,75.0,2.0,6.0,4.0,2.0,3,12.0,9.0,0.0
1,Aaron Ramsey,MF,Wales,31-357,1990,3.0,147.0,6.0,32.0,73.0,...,2.0,0.0,2.0,4.0,1.0,3.0,0,2.0,2.0,1.0
2,Abdelhamid Sabiri,MF,Morocco,26-020,1996,2.0,86.0,5.0,23.0,50.0,...,6.0,33.3,4.0,3.0,1.0,2.0,5,8.0,3.0,0.0
3,Abdelkarim Hassan,DF,Qatar,29-112,1993,3.0,193.0,15.0,83.0,95.0,...,4.0,75.0,1.0,3.0,1.0,2.0,1,8.0,5.0,0.0
4,Abderrazak Hamdallah,FW,Morocco,32-001,1990,0.8,28.0,2.0,2.0,15.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0,0.0,2.0,0.0


In [10]:
def shorter_name(full_name):
    parts = full_name.split()
    if len(parts) == 1:
        return full_name
    elif len(parts) >= 2:
        first_initial = parts[0][0].upper() + '.' if parts[0] else '' 
        last_name = parts[-1].capitalize()
        return f"{first_initial} {last_name}"
    else:
        return full_name

merged_df['player'] = merged_df['player'].apply(shorter_name)


merged_df.head()

,player,position,team,age,birth_year,minutes_90s,touches,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,...,dribbles_vs,dribble_tackles_pct,dribbled_past,blocks,blocked_shots,blocked_passes,interceptions,tackles_interceptions,clearances,errors
0,A. Mooy,MF,Australia,32-094,1990,4.0,255.0,14.0,62.0,168.0,...,8.0,75.0,2.0,6.0,4.0,2.0,3,12.0,9.0,0.0
1,A. Ramsey,MF,Wales,31-357,1990,3.0,147.0,6.0,32.0,73.0,...,2.0,0.0,2.0,4.0,1.0,3.0,0,2.0,2.0,1.0
2,A. Sabiri,MF,Morocco,26-020,1996,2.0,86.0,5.0,23.0,50.0,...,6.0,33.3,4.0,3.0,1.0,2.0,5,8.0,3.0,0.0
3,A. Hassan,DF,Qatar,29-112,1993,3.0,193.0,15.0,83.0,95.0,...,4.0,75.0,1.0,3.0,1.0,2.0,1,8.0,5.0,0.0
4,A. Hamdallah,FW,Morocco,32-001,1990,0.8,28.0,2.0,2.0,15.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0,0.0,2.0,0.0


In [11]:
merged_df = pd.merge(merged_df,df5_qualified,how='inner',left_on='player',right_on='Name')

In [12]:
merged_df_sort = merged_df.sort_values(by='Overall', ascending=False)


cleaned_merged_df = merged_df_sort.drop_duplicates(subset='Name', keep='first')


print(cleaned_merged_df)

             player position       team     age  birth_year  minutes_90s  \
302        L. Messi       FW  Argentina  35-177        1987          7.7   
428  R. Lewandowski       FW     Poland  34-119        1988          4.0   
292       K. Mbappé       FW     France  23-363        1998          6.6   
381          Neymar       MF     Brazil  30-316        1992          3.1   
92       C. Ronaldo       FW   Portugal  37-316        1985          3.2   
..              ...      ...        ...     ...         ...          ...   
54        A. Ounahi       MF    Morocco  22-243        2000          6.3   
194          H. Ito       DF      Japan  23-220        1999          0.5   
118       D. Lovren       DF    Croatia  33-166        1989          6.7   
220    J. Sarmiento       MF    Ecuador  20-185        2002          0.9   
441      R. Colwill       MF      Wales  20-235        2002          0.1   

     touches  touches_def_pen_area  touches_def_3rd  touches_mid_3rd  ...  \
302    460

In [13]:
#Drop unnecessary columns
columns_to_drop = ['touches_def_pen_area','touches_def_3rd','touches_mid_3rd','touches_att_3rd','touches_att_pen_area',
                  'touches_live_ball','dribbles','dribbles_completed_pct','miscontrols', 'dispossessed', 'passes_received',
                   'progressive_passes_received', 'shots_on_target_pct', 'shots_per90', 'shots_on_target_per90',
                  'goals_per_shot_on_target','average_shot_distance','passes_pct', 'passes_total_distance', 
                   'passes_progressive_distance', 'passes_completed_short', 'passes_short', 'passes_pct_short',
                   'passes_completed_medium', 'passes_medium', 'passes_pct_medium','passes_completed_long', 'passes_long', 
                   'passes_pct_long', 'xg_assist', 'pass_xa', 'xg_assist_net', 'assisted_shots','passes_into_final_third', 
                   'passes_into_penalty_area','crosses_into_penalty_area', 'progressive_passes','tackles_def_3rd', 
                   'tackles_mid_3rd', 'tackles_att_3rd'
                  ]
df_dropped = cleaned_merged_df.drop(columns_to_drop, axis=1)


In [14]:
#Drop unnecessary columns
columns_2_drop = ['shots_free_kicks','pens_made','pens_att', 'xg', 'npxg', 'npxg_per_shot', 'xg_net', 'npxg_net',
                  'tackles', 'dribble_tackles', 'dribbles_vs', 'dribble_tackles_pct', 'dribbled_past', 'blocks', 
                  'blocked_shots', 'blocked_passes', 'interceptions', 'tackles_interceptions','clearances', 'errors',
                  'birth_year', 'age', 'ID','Name','Photo','Nationality', 'Flag', 'Potential','Club','Club Logo','Wage',
                  'Special','Preferred Foot','International Reputation','Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 
                  'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 
                  'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 
                  'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 
                  'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 
                  'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 
                  'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position', 
                  'Best Overall Rating', 'Release Clause', 'DefensiveAwareness',
                  ]
data_df = df_dropped.drop(columns_2_drop, axis=1)

In [15]:
data_df.dtypes

player                 object
position               object
team                   object
minutes_90s           float64
touches               float64
dribbles_completed    float64
goals                   int64
shots                   int64
shots_on_target         int64
goals_per_shot        float64
passes_completed      float64
passes                float64
assists                 int64
tackles_won             int64
Age                     int64
Overall                 int64
Value                 float64
dtype: object

In [16]:
# Display Result
data_df

,player,position,team,minutes_90s,touches,dribbles_completed,goals,shots,shots_on_target,goals_per_shot,passes_completed,passes,assists,tackles_won,Age,Overall,Value
302,L. Messi,FW,Argentina,7.7,460.0,15.0,7,27,13,0.11,301.0,372.0,3,3,34,93,78000000.0
428,R. Lewandowski,FW,Poland,4.0,153.0,5.0,2,10,2,0.10,61.0,91.0,1,2,32,92,119500000.0
292,K. Mbappé,FW,France,6.6,325.0,25.0,8,29,11,0.21,173.0,229.0,2,0,22,91,194000000.0
381,Neymar,MF,Brazil,3.1,203.0,6.0,2,11,6,0.09,121.0,153.0,0,0,29,91,129000000.0
92,C. Ronaldo,FW,Portugal,3.2,116.0,0.0,1,10,2,0.00,68.0,90.0,0,0,36,91,45000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,A. Ounahi,MF,Morocco,6.3,328.0,10.0,0,6,1,0.00,216.0,265.0,0,8,21,62,975000.0
194,H. Ito,DF,Japan,0.5,64.0,0.0,0,0,0,NaN,58.0,63.0,0,0,22,62,825000.0
118,D. Lovren,DF,Croatia,6.7,515.0,0.0,0,4,0,0.00,405.0,460.0,1,2,21,60,550000.0
220,J. Sarmiento,MF,Ecuador,0.9,44.0,2.0,0,1,1,0.00,23.0,37.0,0,1,19,59,525000.0


In [ ]:
# Create a Polars DataFrame
df = pl.DataFrame(data_df)
print(type(df))
df.head()

In [ ]:
# Rename the columns
new_column_names = {
    'position_x': 'position',
    'age_x': 'age',
    'team_x': 'team'
}

df_renamed = df.rename(new_column_names)

# Display the result
df_renamed.head()

In [ ]:
# Drop unnecesary values from columns

df_5 = df_renamed.with_columns(pl.col('age').map_elements(lambda value:
                                                     value.split('-')[0]))
df_5.head()

In [ ]:
# Casting data to their corrisponding data type

new_columns = df_5.with_columns(
    pl.col("age").cast(pl.Int64),
    pl.col("touches").cast(pl.Int64),
    pl.col("dribbles_completed").cast(pl.Int64),
    pl.col("passes_completed").cast(pl.Int64),
    pl.col("passes").cast(pl.Int64)
)
new_columns.tail()

In [ ]:
#Drop any null values in the DataFrame
new_columns = new_columns.drop_nulls()

In [ ]:
#sort values depending on ammount of goals
final_df = new_columns.sort(["goals"], descending=True)

In [ ]:
#Display final results
final_df

In [ ]:
#Create DataFame
final_df = pl.DataFrame(final_df)
#Save DataFrame as a csv file
final_df.write_csv('FIFA_stats_cleaned.csv')

